In [1]:
import cv2
import random
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

prev_index_x = None
square_size = 50 
square_x = frame_width // 2 
square_y = frame_height - square_size

right = False
left = False

class Circle:
    def __init__(self, circle_x, circle_y,speed,fallen):
        self.circle_x = circle_x
        self.circle_y = circle_y
        self.speed = speed
        self.fallen = fallen
circles = []

for i in range(3):
    circle_x = random.randint(25,frame_width-25)
    circle_y = random.randint(-50,-25)
    speed = random.randint(15,25)
    circles.append(Circle(circle_x,circle_y,speed,False))

fallen = False

game_over = False

lives = 3
score = 0

while cap.isOpened():
    state, frame = cap.read()

    if state:
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(frame_rgb)

        if not game_over:
            for circle in circles:
                if circle.circle_y >= frame_height+25 and not circle.fallen:
                    score+=1
                    circle.fallen = True

                if circle.fallen:
                    circle.circle_x = random.randint(25,frame_width-25)
                    circle.circle_y = random.randint(-50,-25)
                    circle.speed = random.randint(15,25)
                    circle.fallen = False
                else:
                    circle.circle_y+=circle.speed

                cv2.circle(frame,(circle.circle_x,circle.circle_y),20,(0,0,255),5)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                index_tip = hand_landmarks.landmark[8]
                index_x, index_y = index_tip.x * frame.shape[1], index_tip.y * frame.shape[0]

                if prev_index_x is not None:
                    delta_x = index_x - prev_index_x
                    movement_threshold = 10

                    if abs(delta_x) > movement_threshold:
                        if delta_x > 0:
                            right = True
                            left = False
                        else:
                            left = True
                            right = False

                prev_index_x = index_x

        if not game_over:  
    
            if left:
                square_x-=10
            
            if right:
                square_x+=10

            square_x = max(0,min(frame_width-square_size,square_x))

            cv2.rectangle(frame,(square_x,square_y),(square_x+square_size,square_y+square_size),(255,0,0),-1)

        for circle in circles:
            if circle.circle_x + 25 >= square_x and circle.circle_x - 25 <= square_x + square_size and circle.circle_y + 25 >= square_y and circle.circle_y - 25 <= square_y + square_size:
                lives-=1
                circles.remove(circle)
                circles.append(Circle(random.randint(25,frame_width-25),
                                random.randint(-50,-25),random.randint(15,25),False))

        if lives == 0:
                game_over = True
                cv2.putText(frame, "GAME OVER!", (frame_width // 2 - 100, frame_height // 2), cv2.FONT_ITALIC, 1, (255,0,0), 3)
                circles = []
            
        cv2.putText(frame,f"Score: {score}",(30,50),cv2.FONT_ITALIC,1,(255,0,0),3)
        cv2.putText(frame,f"Lives: {lives}",(frame_width-180,50),cv2.FONT_ITALIC,1,(255,0,0),3)

        cv2.imshow("Avoid", frame)

        k = cv2.waitKey(1)
        if k == ord("q"):
            break
        elif k == ord("r") and game_over:
            prev_index_x = None
            square_size = 50 
            square_x = frame_width // 2 
            right = False
            left = False
            circles = []
            for i in range(3):
                circle_x = random.randint(25,frame_width-25)
                circle_y = random.randint(-50,-25)
                speed = random.randint(15,25)
                circles.append(Circle(circle_x,circle_y,speed,False))
            fallen = False
            game_over = False
            lives = 3
            score = 0
    else:
        break

cap.release()
cv2.destroyAllWindows()
